<a href="https://colab.research.google.com/github/paridhiv21/Customer-Segmentation-Credit-Risk-Modelling/blob/main/msci_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

file_path = 'MSCI data work.xlsx'
xls = pd.read_excel(file_path)

In [ ]:
msc_out = pd.read_excel(file_path, sheet_name='MSCI in & out')

In [ ]:

msc_out.columns = msc_out.iloc[0]

msc_out = msc_out.drop(0)

msc_out.columns = ['Announcement Date', 'Adjustment Date', 'Inclusion Name', 'Inclusion Identifier',
                   'Drop1', 'Announcement Date Ex', 'Adjustment Date Ex', 'Exclusion Name', 'Exclusion Identifier']
msc_out = msc_out[['Announcement Date', 'Adjustment Date', 'Inclusion Name', 'Inclusion Identifier',
                   'Exclusion Name', 'Exclusion Identifier']]


In [ ]:
historical_prices = pd.read_excel(file_path,sheet_name='Data')

In [ ]:

import pandas as pd

msc_out['Announcement Date'] = pd.to_datetime(msc_out['Announcement Date'])

historical_prices['Date'] = pd.to_datetime(historical_prices['Date'])

msc_out['One Month Before'] = msc_out['Announcement Date'] - pd.DateOffset(months=1)
msc_out['29 Days Before'] = msc_out['Announcement Date'] - pd.DateOffset(days=29)
msc_out['Thirty Days Before'] = msc_out['Announcement Date'] - pd.DateOffset(days=30)




In [ ]:
def get_price(identifier, primary_date, alternate_date_1, alternate_date_2):
    price = historical_prices[(historical_prices['Bloom'] == identifier) & (historical_prices['Date'] == primary_date)]['Close Price']
    if price.empty:
        price = historical_prices[(historical_prices['Bloom'] == identifier) & (historical_prices['Date'] == alternate_date_1)]['Close Price']
    if price.empty:
        price = historical_prices[(historical_prices['Bloom'] == identifier) & (historical_prices['Date'] == alternate_date_2)]['Close Price']
    return price.iloc[0] if not price.empty else None

In [ ]:
initial_capital = 1000000
updated_capital = initial_capital
profits = []


In [ ]:
for date in sorted(msc_out['Announcement Date'].dropna().unique()):
    date_data = msc_out[msc_out['Announcement Date'] == date]
    number_of_transactions = date_data['Inclusion Identifier'].notna().sum() + date_data['Exclusion Identifier'].notna().sum()
    capital_per_transaction = initial_capital / number_of_transactions if number_of_transactions else 0

    date_profit = 0

    for _, row in date_data.iterrows():
        if pd.notna(row['Inclusion Identifier']):
            buy_price = get_price(row['Inclusion Identifier'], row['One Month Before'], row['29 Days Before'], row['Thirty Days Before'])
            sell_price = get_price(row['Inclusion Identifier'], row['Announcement Date'], row['Announcement Date'], row['Announcement Date'])
            if buy_price and sell_price:
                shares = capital_per_transaction / buy_price
                profit = shares * (sell_price - buy_price)
                date_profit += profit
                profits.append({'Date': date, 'Type': 'Inclusion', 'Profit': profit})
                print(f"Buying: {shares} shares of {row['Inclusion Name']} at {buy_price}, selling at {sell_price}, profit: {profit}")

        if pd.notna(row['Exclusion Identifier']):
            short_price = get_price(row['Exclusion Identifier'], row['One Month Before'], row['29 Days Before'], row['Thirty Days Before'])
            cover_price = get_price(row['Exclusion Identifier'], row['Announcement Date'], row['Announcement Date'], row['Announcement Date'])
            if short_price and cover_price:
                shares = capital_per_transaction / short_price
                profit = shares * (short_price - cover_price)
                date_profit += profit
                profits.append({'Date': date, 'Type': 'Exclusion', 'Profit': profit})
                print(f"Shorting: {shares} shares of {row['Exclusion Name']} at {short_price}, covering at {cover_price}, profit: {profit}")

    initial_capital += date_profit

profits_df = pd.DataFrame(profits)
profit_summary = profits_df.groupby('Date').agg(Total_Profit=('Profit', 'sum')).reset_index()

total_accumulated_profit = profits_df['Profit'].sum()

print("Updated Profit Summary by Announcement Date:")
print(profit_summary)
print(f"Total Accumulated Profit: {total_accumulated_profit}")


Buying: 367.0532961385993 shares of Biocon at 340.55, selling at 337.25, profit: -1211.2758772573818
Shorting: 2714.440825190011 shares of Ashok Leyland at 46.05, covering at 48.75, profit: -7328.990228013037
Buying: 271.7686705076639 shares of Indraprastha Gas at 459.95, selling at 473.0, profit: 3546.581150125017
Shorting: 1331.7707223524399 shares of M & M Financial Svcs at 93.86, covering at 107.83, profit: -18604.836991263583
Buying: 452.43955407557553 shares of Jubilant Foodworks at 276.28, selling at 339.81, profit: 28743.484870421325
Shorting: 3481.8941504178274 shares of Tata Power Co at 35.9, covering at 30.3, profit: 19498.607242339825
Buying: 409.7688903458449 shares of Tata Consumer Products at 305.05, selling at 356.35, profit: 21021.14407474185
Buying: 105.24986317517786 shares of Torrent Pharma at 1187.65, selling at 1202.08, profit: 1518.7555256177993
Buying: 52.930692642139086 shares of Acc at 1521.85, selling at 1670.15, profit: 7849.621718829236
Shorting: 6.62479796